In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/akinrobotics

/content/drive/My Drive/akinrobotics


In [ ]:
import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re # regular expression libary.
import nltk # Natural Language toolkit
nltk.download("stopwords")  #downloading stopwords
nltk.download('punkt')
from nltk import word_tokenize,sent_tokenize
nltk.download('wordnet')
import nltk as nlp
from nltk.corpus import stopwords

import joblib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,precision_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
data = pd.read_csv("7allV03.csv")

In [ ]:
data

,category,text
0,siyaset,3 milyon ile ön seçim vaadi mhp nin 10 olağan...
1,siyaset,mesut_yılmaz yüce_divan da ceza alabilirdi pr...
2,siyaset,disko lar kaldırılıyor başbakan_yardımcısı ar...
3,siyaset,sarıgül anayasa_mahkemesi ne gidiyor mustafa_...
4,siyaset,erdoğan idamın bir haklılık sebebi var demek ...
...,...,...
4895,teknoloji,iphone lara geri dönüyor ios 6 sürümüyle tele...
4896,teknoloji,muslukta devrim sadece elimizi yıkadığımız mu...
4897,teknoloji,halka iyi anlatılmalı bilgi_teknolojileri ile...
4898,teknoloji,çöpe gidiyorlar apple 775 bin uygulamayla app...


In [ ]:
data['category'] = data['category'].replace({'dunya': 0, 'ekonomi': 1, 'kultur': 2, 'saglik': 3, 'siyaset': 4, 'spor': 5, 'teknoloji': 6})

In [ ]:
df_copy = data

In [ ]:
text_list=[]

for text in df_copy.text:
    text = text.lower()
    text = re.sub("[^abcçdefgğhıijklmnoöprsştuüvyz]"," ",text)
    text=nltk.word_tokenize(text)
    text =[word for word in text if not word in set(stopwords.words("turkish"))]
    lemma=nlp.WordNetLemmatizer()
    text=[lemma.lemmatize(word) for word in text]
    text=" ".join(text)
    text_list.append(text)

In [ ]:
df_copy

,category,text
0,4,3 milyon ile ön seçim vaadi mhp nin 10 olağan...
1,4,mesut_yılmaz yüce_divan da ceza alabilirdi pr...
2,4,disko lar kaldırılıyor başbakan_yardımcısı ar...
3,4,sarıgül anayasa_mahkemesi ne gidiyor mustafa_...
4,4,erdoğan idamın bir haklılık sebebi var demek ...
...,...,...
4895,6,iphone lara geri dönüyor ios 6 sürümüyle tele...
4896,6,muslukta devrim sadece elimizi yıkadığımız mu...
4897,6,halka iyi anlatılmalı bilgi_teknolojileri ile...
4898,6,çöpe gidiyorlar apple 775 bin uygulamayla app...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

max_features=500

count_vectorizer=CountVectorizer(max_features=max_features)

sparce_matrix=count_vectorizer.fit_transform(text_list).toarray()

In [ ]:
sparce_matrix.shape

(4900, 500)

In [ ]:
sparce_matrix[0:10,0:20]

array([[0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

labels=le.fit_transform(df_copy.category)

In [ ]:
X = sparce_matrix
y = labels

In [ ]:
data=pd.DataFrame(count_vectorizer.get_feature_names_out(),columns=["Words"])

In [ ]:
df_copy

,category,text
0,4,3 milyon ile ön seçim vaadi mhp nin 10 olağan...
1,4,mesut_yılmaz yüce_divan da ceza alabilirdi pr...
2,4,disko lar kaldırılıyor başbakan_yardımcısı ar...
3,4,sarıgül anayasa_mahkemesi ne gidiyor mustafa_...
4,4,erdoğan idamın bir haklılık sebebi var demek ...
...,...,...
4895,6,iphone lara geri dönüyor ios 6 sürümüyle tele...
4896,6,muslukta devrim sadece elimizi yıkadığımız mu...
4897,6,halka iyi anlatılmalı bilgi_teknolojileri ile...
4898,6,çöpe gidiyorlar apple 775 bin uygulamayla app...


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
print("x_train",X_train.shape)
print("x_test",X_test.shape)
print("y_train",y_train.shape)
print("y_test",y_test.shape)

x_train (3920, 500)
x_test (980, 500)
y_train (3920,)
y_test (980,)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



model = LogisticRegression()


model.fit(X_train, y_train)


y_pred = model.predict(X_test)
joblib.dump(model, 'logistic_regression.joblib')

accuracy = accuracy_score(y_test, y_pred)
print("Doğruluk Skoru:", accuracy)




Doğruluk Skoru: 0.8295918367346938
